In [32]:
import yaml
from sqlalchemy import create_engine
from sqlalchemy import inspect
import data_cleaning as dc
import data_extraction as de
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
import pandas as pd

dab = DatabaseConnector('db_creds.yaml')
dab.init_db_engine()
#List the tables
tables = dab.list_db_tables()
#prints tables
print(tables)




['legacy_store_details', 'legacy_users', 'orders_table']


In [33]:
de = DataExtractor()
original_df = de.extract_user_data()

#Copy the data from the table into a Pandas DF
#pandaDF = dab.read_db_creds()
#for col in pandaDF.columns:
    #print(col)

#db_yaml_file='db_creds.yaml'
#db_connector=DatabaseConnector(db_yaml_file)

pd.set_option('display.max_columns', None)
legacy_users = de.read_rds_table('legacy_users')
legacy_users.head(5)


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [34]:
legacy_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [35]:
legacy_users['country'].value_counts()

United Kingdom    9371
Germany           4708
United States     1205
NULL                21
GMRBOMI0O1           1
7ZNO5EBALT           1
3518UD5CE8           1
RQRB7RMTAD           1
PNRMPSYR1J           1
5EFAFD0JLI           1
YOTSVPRBQ7           1
50KUU3PQUF           1
EWE3U0DZIV           1
XN9NGL5C0B           1
S0E37H52ON           1
XGI7FM0VBJ           1
AJ1ENKS3QL           1
I7G4DMDZOZ           1
T4WBZSW0XI           1
Name: country, dtype: int64

In [36]:
# There are many that are not actually a country. There are only three correct values. 

a = ['United Kingdom','United States','Germany']
cleaned_users = legacy_users[legacy_users.country.isin(a)]
cleaned_users['country'].value_counts()

United Kingdom    9371
Germany           4708
United States     1205
Name: country, dtype: int64

In [37]:
cleaned_users['country_code'].value_counts()

GB     9365
DE     4708
US     1205
GGB       6
Name: country_code, dtype: int64

In [38]:
cleaned_users['country_code'].replace(to_replace='GGB',value='GB', inplace=True)
cleaned_users['country_code'].value_counts()

C:\Users\yurir\AppData\Local\Temp\ipykernel_28012\1802846815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_users['country_code'].replace(to_replace='GGB',value='GB', inplace=True)


GB    9371
DE    4708
US    1205
Name: country_code, dtype: int64

In [43]:
dobs = print(cleaned_users['date_of_birth'])


0        1990-09-30
1        1940-12-01
2        1995-08-02
3        1972-09-23
4        1952-12-20
            ...    
15315    1943-08-09
15316    1948-08-20
15317    1940-10-09
15318    1952-06-04
15319    1994-03-27
Name: date_of_birth, Length: 15284, dtype: object


In [44]:
#Very large data set with, lets see if value count will work? 

dobs = print(cleaned_users['date_of_birth'].value_counts())

1958-02-02    6
1978-01-31    5
1956-07-16    5
1961-03-27    5
1951-01-09    5
             ..
1938-12-22    1
2005-03-28    1
1963-04-26    1
1964-07-13    1
1994-03-27    1
Name: date_of_birth, Length: 11344, dtype: int64


In [49]:
#need another way to represent data
#will count what how many values have varring sets of dob

dobs=cleaned_users['date_of_birth']
dobs.map(lambda calc: len(calc)).value_counts()


10    15268
15        6
16        5
12        4
11        1
Name: date_of_birth, dtype: int64

In [51]:
cleaned_users = cleaned_users[dobs.apply(lambda x: len(x)==10)]
dobs=cleaned_users['date_of_birth']
dobs.map(lambda calc: len(calc)).value_counts()

C:\Users\yurir\AppData\Local\Temp\ipykernel_28012\1233488452.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_users = cleaned_users[dobs.apply(lambda x: len(x)==10)]


10    15268
Name: date_of_birth, dtype: int64

In [55]:
phone_series = cleaned_users['phone_number']
phone_series2 = phone_series.apply(lambda x: ''.join([i for i in x if str.isnumeric(i)])[-10:])
cleaned_users['phone_number']=phone_series2
print(phone_series2)

0        0047905356
1        1614960674
2        1214960340
3        3069990871
4        1214960225
            ...    
15315    0292018946
15316    1144960977
15317    0298408192
15318    2397113836
15319    1314960870
Name: phone_number, Length: 15268, dtype: object


In [2]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    return card_data

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [3]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    return card_data['card_provider'].value_counts()

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
NULL                             11
OGJTXI6X1H                        1
BU9U947ZGV                        1
UA07L7EILH                        1
XGZBYBYGUW                        1
DLWF2HANZF                        1
1M38DYQTZV                        1
WJVMUO4QX6                        1
DE488ORDXY                        1
5CJH7ABGDR                        1
JCQMU8FN85                        1
TS8A81WFXV                        1
JRPRLPIBZ2                        1
NB71VBAHJE                        1
5MFWFBZRM9                        1
Name: card_provider, dtype: int64

In [6]:
'''We have 11 Null values and the rest are made up.'''

import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    '''Card cleaning'''
    b = ['VISA 16 digit','JCB 16 digit','VISA 13 digit','JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']
    card_data = card_data[card_data.card_provider.isin(b)]
    return card_data['card_provider'].value_counts()

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: card_provider, dtype: int64

In [7]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    '''Card Provider cleaning'''
    b = ['VISA 16 digit','JCB 16 digit','VISA 13 digit','JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']
    card_data = card_data[card_data.card_provider.isin(b)]
    '''Card number cleaning'''
    null_map = card_data.applymap(lambda x: isinstance(x, (int, float)))['card_number']
    return card_data.iloc[null_map.values]

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [9]:
'''From 15309 rows × 4 columns to 12909 rows × 4 columns '''
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    return card_data['expiry_date'].value_counts()

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


03/28         163
12/29         163
01/28         162
12/26         152
05/25         152
             ... 
VNLNMWPJII      1
WDWMN9TU45      1
ACT9K6ECRJ      1
8YJ3TYH6Z5      1
2ANT8LW3I5      1
Name: expiry_date, Length: 136, dtype: int64

In [10]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    '''Clean date data'''
    card_data['expiry_date'] = pd.to_datetime(card_data['expiry_date'], errors='coerce')
    card_data = card_data.loc[card_data['expiry_date'].notnull()]
    return card_data['expiry_date'].value_counts()

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


2032-10-01    148
2032-01-01    135
2032-06-01    127
2032-09-01    125
2032-05-01    125
2032-08-01    124
2032-07-01    118
2032-03-01    117
2032-04-01    112
2032-02-01    103
2032-11-01     78
Name: expiry_date, dtype: int64

In [11]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    '''Clean date data'''
    card_data['expiry_date'] = pd.to_datetime(card_data['expiry_date'], format='%m/%y', errors='coerce')
    # Filter out rows where 'expiry_date' is not a valid date
    card_data = card_data.loc[card_data['expiry_date'].notnull()]
    # Convert 'expiry_date' column back to the 'MM/YY' format
    card_data['expiry_date'] = card_data['expiry_date'].dt.strftime('%m/%y')
    # Return the value counts of the remaining valid dates
    return card_data['expiry_date'].value_counts()

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

number of dataframes or page in pdf:
307


03/28    163
12/29    163
01/28    162
05/25    152
12/26    152
        ... 
02/32    103
02/23     94
09/25     92
11/32     78
11/22     37
Name: expiry_date, Length: 121, dtype: int64

In [15]:
import tabula
import pandas as pd
 
def retrieve_pdf_data(url: str):
    '''takes PDF url as input and outputs PDF data as table'''
    dfs =tabula.read_pdf(url,pages='all')
    card_data=pd.concat(dfs,ignore_index=True)
    print('number of dataframes or page in pdf:')
    print(len(dfs))
    print('Number of rows after before Card Provider cleaning:', len(card_data))
    '''Card Provider cleaning'''
    b = ['VISA 16 digit','JCB 16 digit','VISA 13 digit','JCB 15 digit', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard']
    card_data = card_data[card_data.card_provider.isin(b)]
    print('Number of rows after Card Provider cleaning:', len(card_data))

    '''Clean expiry_date col'''#May not need as makes no difference. 
    card_data['expiry_date'] = pd.to_datetime(card_data['expiry_date'], format='%m/%y', errors='coerce')
    # Filter out rows where 'expiry_date' is not a valid date
    card_data = card_data.loc[card_data['expiry_date'].notnull()]
    # Convert 'expiry_date' column back to the 'MM/YY' format
    card_data['expiry_date'] = card_data['expiry_date'].dt.strftime('%m/%y')
    print('Number of rows after expiry_date cleaning:', len(card_data))

    '''Card number cleaning'''
    null_map = card_data.applymap(lambda x: isinstance(x, (int, float)))['card_number']
    card_data_cleaned = card_data.iloc[null_map.values]
    print('Number of rows after card number cleaning:', len(card_data_cleaned))
    return card_data_cleaned

retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')


number of dataframes or page in pdf:
307
Number of rows after before Card Provider cleaning: 15309
Number of rows after Card Provider cleaning: 15284
Number of rows after expiry_date cleaning: 15284
Number of rows after card number cleaning: 12909


,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [32]:
import requests

def list_number_of_stores1():
        headers = {
            "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
        }

        response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
        return response

def list_number_of_stores2():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    print(response.json())  # Print the JSON response for debugging
    return response     


list_number_of_stores2()


{'statusCode': 200, 'number_stores': 451}


<Response [200]>

In [36]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
        if i % 10 == 0:
            print(f"Collected record {i}")
    df = pd.DataFrame(stores_data)
    return df        

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)    


Collected record 0
Collected record 10
Collected record 20
Collected record 30
Collected record 40
Collected record 50
Collected record 60
Collected record 70
Collected record 80
Collected record 90
Collected record 100
Collected record 110
Collected record 120
Collected record 130
Collected record 140
Collected record 150
Collected record 160
Collected record 170
Collected record 180
Collected record 190
Collected record 200
Collected record 210
Collected record 220
Collected record 230
Collected record 240
Collected record 250
Collected record 260
Collected record 270
Collected record 280
Collected record 290
Collected record 300
Collected record 310
Collected record 320
Collected record 330
Collected record 340
Collected record 350
Collected record 360
Collected record 370
Collected record 380
Collected record 390
Collected record 400
Collected record 410
Collected record 420
Collected record 430
Collected record 440
Collected record 450


,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,None,None
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
446,446,"Täschestraße 25\n39039 Nördlingen, Kirchlengern",52.2,None,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
447,447,K0ODETRLS3,K8CXLZDP07,UXMWDMX1LC,3VHFDNP8ET,9D4LK7X4LZ,D23PCWSM6S,36IIMAQD58,NN04B3F6UQ,JZP8MIJTPZ,B3EH2ZGQAV,1WZB1TE1HL
448,448,"Studio 8\nMoss mall\nWest Linda\nM0E 6XR, High...",51.62907,None,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe
449,449,"Baumplatz 6\n80114 Kötzting, Bretten",49.03685,None,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe


In [37]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
        if i % 10 == 0:
            print(f"Collected record {i}")
    df = pd.DataFrame(stores_data)
    return df        

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key).info()  

Collected record 0
Collected record 10
Collected record 20
Collected record 30
Collected record 40
Collected record 50
Collected record 60
Collected record 70
Collected record 80
Collected record 90
Collected record 100
Collected record 110
Collected record 120
Collected record 130
Collected record 140
Collected record 150
Collected record 160
Collected record 170
Collected record 180
Collected record 190
Collected record 200
Collected record 210
Collected record 220
Collected record 230
Collected record 240
Collected record 250
Collected record 260
Collected record 270
Collected record 280
Collected record 290
Collected record 300
Collected record 310
Collected record 320
Collected record 330
Collected record 340
Collected record 350
Collected record 360
Collected record 370
Collected record 380
Collected record 390
Collected record 400
Collected record 410
Collected record 420
Collected record 430
Collected record 440
Collected record 450
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [45]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    return df

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['staff_numbers'].value_counts() 

20            18
33            17
28            16
29            16
7             16
              ..
76             1
58             1
56             1
2429OB3LMM     1
D23PCWSM6S     1
Name: staff_numbers, Length: 114, dtype: int64

In [46]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Convert staff_numbers column to numeric and drop non-numeric values
    df['staff_numbers'] = pd.to_numeric(df['staff_numbers'], errors='coerce')
    df_cleaned = df.dropna(subset=['staff_numbers'])
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['staff_numbers'].value_counts()

20.0     18
33.0     17
29.0     16
28.0     16
7.0      16
         ..
71.0      1
128.0     1
76.0      1
58.0      1
61.0      1
Name: staff_numbers, Length: 101, dtype: int64

In [47]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]


    
def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    
    # Convert staff_numbers column to numeric and drop non-numeric values
    df['staff_numbers'] = pd.to_numeric(df['staff_numbers'], errors='coerce')
    df_cleaned = df.dropna(subset=['staff_numbers'])
    return df_cleaned['staff_numbers'].value_counts()

num_stores = list_number_of_stores()
API_key = {'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
staff_counts = retrieve_store_data(num_stores, get_stores_endpoint, API_key)
print(staff_counts)


20.0     18
33.0     17
29.0     16
28.0     16
7.0      16
         ..
71.0      1
128.0     1
76.0      1
58.0      1
61.0      1
Name: staff_numbers, Length: 101, dtype: int64


In [51]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Convert staff_numbers column to numeric and drop non-numeric values
    df['staff_numbers'] = pd.to_numeric(df['staff_numbers'], errors='coerce')
    df_cleaned = df.dropna(subset=['staff_numbers'])
    #The .loc accessor ensures that the changes made to the subset of the DataFrame will be propagated back to the original DataFrame.
    df_cleaned.loc[:, 'staff_numbers'] = df_cleaned['staff_numbers'].astype(int).copy()
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['staff_numbers'].value_counts()

C:\Users\yurir\AppData\Local\Temp\ipykernel_24448\3007001750.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:, 'staff_numbers'] = df_cleaned['staff_numbers'].astype(int).copy()
C:\Users\yurir\AppData\Local\Temp\ipykernel_24448\3007001750.py:26: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[:, 'staff_numbers'] = df_cleaned['staff_numbers'].astype(int).copy()


20     18
33     17
29     16
28     16
7      16
       ..
71      1
128     1
76      1
58      1
61      1
Name: staff_numbers, Length: 101, dtype: int64

In [52]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['staff_numbers'].value_counts()

20     18
33     17
29     16
28     16
7      16
       ..
71      1
128     1
76      1
58      1
61      1
Name: staff_numbers, Length: 101, dtype: int64

In [53]:
'''Does previous code also get rid?'''
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    return df

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['country_code'].value_counts() 

GB            265
DE            141
US             34
NULL            3
YELVM536YT      1
FP8DLXQVGH      1
HMHIFNLOBN      1
F3AO8V2LHU      1
OH20I92LX3      1
OYVW925ZL8      1
B3EH2ZGQAV      1
Name: country_code, dtype: int64

In [54]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['country_code'].value_counts()

GB    263
DE    139
US     33
Name: country_code, dtype: int64

In [3]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    df_cleaned['continent'] = df_cleaned['continent'].replace('eeEurope', 'Europe').replace('eeAmerica', 'America')
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['continent'].value_counts()

Europe     402
America     33
Name: continent, dtype: int64

In [6]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    df_cleaned['continent'] = df_cleaned['continent'].replace('eeEurope', 'Europe').replace('eeAmerica', 'America')
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['lat'].value_counts()

N/A    1
Name: lat, dtype: int64

In [7]:
'''Need to remove the column lat which only have nul values'''

import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned = df.drop('lat', axis=1)
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    df_cleaned['continent'] = df_cleaned['continent'].replace('eeEurope', 'Europe').replace('eeAmerica', 'America')
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key).head()

,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,None,None
1,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [8]:
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned = df.drop('lat', axis=1)
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    df_cleaned['continent'] = df_cleaned['continent'].replace('eeEurope', 'Europe').replace('eeAmerica', 'America')
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['opening_date'].value_counts()

2004-01-23     3
2014-05-21     2
1998-12-19     2
1998-09-29     2
2006-10-03     2
              ..
May 2003 27    1
1997-02-07     1
2003-04-07     1
2006-10-18     1
2001-05-12     1
Name: opening_date, Length: 427, dtype: int64

In [9]:
'''Need to keep the unorganised dates can sort out another time in SQL'''
import requests
import pandas as pd

def list_number_of_stores():
    headers = {
        "x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"
    }
    response = requests.get("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores", headers=headers)
    return response.json()["number_stores"]

def retrieve_store_data(number_stores, retrieve_store_endpoint, header):
    stores_data = []
    for i in range(number_stores):
        store_url = retrieve_store_endpoint+str(i)
        try:
            response = requests.get(store_url, headers=header)
        except:
            print("there was an error!")    
        stores_data.append(response.json())
    df = pd.DataFrame(stores_data)
    '''Clean Staff numbers'''
    # Copy the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    # Convert staff_numbers column to numeric and drop non-numeric values
    df_cleaned = df.drop('lat', axis=1)
    df_cleaned['staff_numbers'] = pd.to_numeric(df_cleaned['staff_numbers'], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=['staff_numbers'])
    df_cleaned['staff_numbers'] = df_cleaned['staff_numbers'].astype(int)
    df_cleaned['continent'] = df_cleaned['continent'].replace('eeEurope', 'Europe').replace('eeAmerica', 'America')
    return df_cleaned

num_stores = list_number_of_stores()
API_key={'x-api-key':'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
get_stores_endpoint='https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
retrieve_store_data(num_stores, get_stores_endpoint, API_key)['locality'].value_counts()

Chapletown          14
Belper              13
Bushey              12
Exeter              11
Arbroath            10
                    ..
Redding              1
Wittenau             1
Fairview Heights     1
Brierley Hill        1
Westchester          1
Name: locality, Length: 116, dtype: int64